In [20]:
import json
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pickle 

/tmp/ipykernel_5617/923370504.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open("./dataset/json/mini_publaynat_rnd_walk_dist_veclen50.json", "r") as f:
    dataset = json.load(f)

In [4]:
print("Первый x:\n", dataset["x"][0])
print("Последний, x:\n", dataset["x"][-1])
print("Размер x:\t", len(dataset["x"][0]))
print("Первый y:\t", dataset["y"][0])
print("Последний y:\t", dataset["y"][0])
print("Кол-во записей:\t", len(dataset["x"]))

Первый x:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Последний, x:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Размер x:	 50
Первый y:	 1
Последний y:	 1
Кол-во записей:	 4958


In [112]:

X = np.array(dataset["x"])
y = np.array(dataset["y"])


In [113]:
y = y[X.sum(1)!=0]
X = X[X.sum(1)!=0, :]


In [120]:
X = np.array([x/x.sum() for x in X])
X

array([[0.        , 0.        , 0.        , ..., 0.03030303, 0.06060606,
        0.06060606],
       [0.01546392, 0.01546392, 0.        , ..., 0.01546392, 0.        ,
        0.05154639],
       [0.01632653, 0.01632653, 0.        , ..., 0.        , 0.06122449,
        0.06122449],
       ...,
       [0.04482759, 0.04482759, 0.04482759, ..., 0.02413793, 0.02068966,
        0.01724138],
       [0.        , 0.        , 0.        , ..., 0.03255814, 0.00930233,
        0.00930233],
       [0.        , 0.        , 0.06666667, ..., 0.06666667, 0.06666667,
        0.        ]])

In [121]:
len(y), len(X)

(3505, 3505)

In [122]:
keys = np.unique(y)
Y_keys = dict()
for i, key in enumerate(keys):
    zeros = np.zeros((len(keys)))
    zeros[i] = 1
    Y_keys[key] = zeros
Y_keys
Y = np.array([Y_keys[yi] for yi in y] )
Y_keys, Y

({1: array([1., 0., 0., 0., 0.]),
  2: array([0., 1., 0., 0., 0.]),
  3: array([0., 0., 1., 0., 0.]),
  4: array([0., 0., 0., 1., 0.]),
  5: array([0., 0., 0., 0., 1.])},
 array([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]]))

In [123]:
X_clus_list = []
Y_clus_list = []
for i, item in enumerate(Y_keys.keys()):
    X_clus_list.append(X[Y[:,i]==1,:])
    Y_clus_list.append(Y[Y[:,i]==1,:])

In [124]:
balans_len = [len(yc) for yc in Y_clus_list]
min_len = min(balans_len)
min_len, balans_len

(442, [849, 442, 811, 804, 599])

In [125]:
X = np.concatenate([xc[:min_len] for xc in X_clus_list])
Y = np.concatenate([yc[:min_len] for yc in Y_clus_list])

In [228]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=49)

In [229]:
i = 0
aas = [0.00001]
miis = [2000]
modds = [( 50, 50, 50, 50)]
df = pd.DataFrame(columns = ['alpha','max_iter','model', 'train_acc','test_acc','train_time'])
for a in aas:
    for mi in miis:
        for mod in modds:
            st = time()
            mlp = MLPClassifier(hidden_layer_sizes=mod, alpha=a, max_iter=mi)
            mlp.fit(X_train, y_train)
            end = time() - st
            
            acc_tr = accuracy_score(y_train, mlp.predict(X_train)) # Train Accuracy
            acc = accuracy_score(y_test, mlp.predict(X_test)) # Test Accuracy
            df.loc[i] = [a, mi, mod, acc_tr,acc,end]
            i=i+1

In [230]:
df # Results
# 0	0.00001	2000	(50, 50, 50, 50)	0.765271	0.187783	9.691846 (43)
# 	0.00001	2000	(50, 50, 50, 50)	0.75	0.192308	9.430116 (44)
# 0.00001	2000	(50, 50, 50, 50)	0.616516	0.199095	10.068232 (45)
# 0.00001	2000	(50, 50, 50, 50)	0.796946	0.20362	6.908683 (46)

,alpha,max_iter,model,train_acc,test_acc,train_time
0,0.00001,2000,"(50, 50, 50, 50)",0.75905,0.190045,7.403339


In [156]:
df["train_acc"].max()
df[df["train_acc"] == df["train_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
0,0.00001,5000,"(100, 50, 5, 5)",0.554864,0.162896,9.305634


In [157]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
0,0.00001,5000,"(100, 50, 5, 5)",0.554864,0.162896,9.305634


In [153]:
df.sort_values("test_acc",  ascending=False)[:15]

,alpha,max_iter,model,train_acc,test_acc,train_time
8,0.00001,20000,"(50, 50, 50, 50)",0.785633,0.221719,7.943425
6,0.00001,10000,"(50, 50, 50, 50, 50, 50)",0.701923,0.199095,8.354873
4,0.00001,10000,"(50, 50, 50, 50)",0.717195,0.196833,10.358008
9,0.00001,20000,"(50, 50, 50, 50, 50)",0.624434,0.194570,4.590786
1,0.00001,5000,"(50, 50, 50, 50, 50)",0.673077,0.187783,8.126346
10,0.00001,20000,"(50, 50, 50, 50, 50, 50)",0.536199,0.171946,4.438486
2,0.00001,5000,"(50, 50, 50, 50, 50, 50)",0.606900,0.160633,5.626406
5,0.00001,10000,"(50, 50, 50, 50, 50)",0.657805,0.158371,9.126492
0,0.00001,5000,"(50, 50, 50, 50)",0.537330,0.151584,10.171808
11,0.00001,20000,"(50, 50, 50, 50, 50, 50, 50)",0.410068,0.131222,4.411276


In [101]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
169,1.0,500,"(20, 20, 20, 20, 20)",0.816742,0.377828,1.473909


In [136]:
df[df["test_acc"] == df["test_acc"].max()]

,alpha,max_iter,model,train_acc,test_acc,train_time
34,0.001,20000,"(50, 5, 10, 15, 20)",0.44457,0.192308,11.896763


In [148]:
Y_clus_list = []
for i, item in enumerate(Y_keys.keys()):
    Y_clus_list.append(y_train[y_train[:,i]==1,:])
balans_len = [len(yc) for yc in Y_clus_list]
balans_len

[339, 365, 357, 349, 358]

In [222]:
# save the model 
filename = 'model-2.sav'
pickle.dump(mlp, open(filename, 'wb')) 